In [ ]:
import os
import json
import pandas as pd

# Directories containing JSON files
directories = ['2018','2019','2020','2021','2022','2023']

# List to store extracted data
extracted_data = []

# Iterate through each directory
for directory in directories:
    # Iterate through each file in the current directory
    for filename in os.listdir(directory):
        if filename.endswith('.json'):  # Ensure it's a JSON file
            filepath = os.path.join(directory, filename)
            
            # Open and load the JSON file
            with open(filepath, 'r', encoding="utf-8") as file:
                data = json.load(file)
                
                # Extract the relevant part of the JSON
                response = data.get('abstracts-retrieval-response', {})
                
                # Extract specific fields
                title = response.get('coredata', {}).get('dc:title', None)
                publicationName = response.get('coredata', {}).get('prism:publicationName', None)
                abstract = response.get('item', {}).get('bibrecord', {}).get('head', {}).get('abstracts', None)
                
                subject_area_list = response.get('subject-areas', {}).get('subject-area', [])
                subjectArea = [item.get('@abbrev', None) for item in subject_area_list if '@abbrev' in item]


                # publication date
                date = response.get('item', {}).get("ait:process-info", {}).get("ait:date-sort",{})
                day = date.get("@day")
                year = date.get("@year")
                month = date.get('@month')
                format_date = f"{day}/{month}/{year}"

                # Extract author keywords and combine into a single string
                # Extract author keywords and combine into a single string
                auth_keywords = response.get('authkeywords', {})
                if isinstance(auth_keywords, dict):  # Ensure it's a dictionary
                    keywords_list = auth_keywords.get('author-keyword', [])
                    if isinstance(keywords_list, list):  # Ensure 'author-keyword' is a list
                        combined_keywords = ",".join(
                        [kw.get('$', '') for kw in keywords_list if isinstance(kw, dict)]
                        )
                else:
                    combined_keywords = None  # Handle cases where auth_keywords is not a dictionary

                # Append the extracted data as a dictionary
                extracted_data.append({
                    'title': title,
                    'publicationName': publicationName,
                    'abstract' : abstract,
                    'keywords': combined_keywords,
                    'subjectArea': subjectArea,
                    'publication_date': format_date
                })

# Convert the extracted data into a DataFrame
df = pd.DataFrame(extracted_data)
def change(x):
    x=  set(x)
    result = ""
    for area in x:
        result += area + ","
    
    return result[:-1]
df["subjectArea"] = df["subjectArea"].apply(change)

In [6]:
df

,title,publicationName,abstract,keywords,subjectArea,publication_date
0,Public health and international epidemiology f...,"Radiology in Global Health: Strategies, Implem...",None,None,MEDI,31/12/2018
1,Flexible Printed Active Antenna for Digital Te...,Progress in Electromagnetics Research Symposium,"© 2018 The Institute of Electronics, Informati...",None,ENGI,31/12/2018
2,Parametric study of hydrogen production via so...,Chemical Engineering Science,© 2018 Elsevier LtdComputational fluid dynamic...,Circulating fluidized bed Computational fluid ...,CHEM,31/12/2018
3,Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science,© 2018 Elsevier B.V. A superhydrophobic/supero...,Encapsulation Fluoroalkylsilane Natural rubber...,CHEM,31/12/2018
4,Electrochemical impedance-based DNA sensor usi...,Analytica Chimica Acta,© 2018 Elsevier B.V. A label-free electrochemi...,acpcPNA Electrochemical impedance spectroscopy...,CHEM,31/12/2018
...,...,...,...,...,...,...
16818,Long-chain bio-olefins production via oxidativ...,Catalysis Today,© 2021 Elsevier B.V.Long-chain α-olefins (≥ C1...,Long-chain olefins Mesoporous KIT-6 Oleic acid...,CENG,01/01/2023
16819,Recent Developments and Applications of Microf...,Critical Reviews in Analytical Chemistry,"© 2021 Taylor & Francis Group, LLC.Nowadays, f...",Biological hazards chemical hazards food conta...,CHEM,01/01/2023
16820,"Social justice, education and peacebuilding: c...",Compare,© 2021 The Author(s). Published by Informa UK ...,conflict Education peacebuilding social justic...,SOCI,01/01/2023
16821,Effects of black soldier fly (Hermetia illucen...,Journal of Applied Aquaculture,© 2021 Taylor & Francis.The effects of replaci...,Anabas testudineus Black soldier fly fish meal...,ENVI,01/01/2023


In [12]:
df.dropna(inplace=True) #axis=0 

In [4]:
df.shape

(13572, 6)

In [8]:
df.groupby('subjectArea').describe()

title                                                            \
            count unique                                                top   
subjectArea                                                                   
AGRI          899    899  Penicillium Imranianum, a new species from the...   
ARTS          171    171  The long reach of English law: A case of incid...   
BIOC          934    933  The effects of beliefs, knowledge, and attitud...   
BUSI          280    280  The critical factors of research and innovatio...   
CENG          581    581  Fibrous platelet carbon nanofibers-silica fibe...   
CHEM          825    825  Comparison of soil composition between farmlan...   
COMP          870    869  Deep multispectral painting reproduction via m...   
DECI           17     17  BEstream: Batch Capturing with Elliptic Functi...   
DENT          233    233  Pediatric cleft palate patients show a 3- to 5...   
EART          183    183  Acid volatile sulphide estimation using spatia...   
ECON          104    104  The impact of financial and trade integration ...   
ENER          391    391  Analysis of a solid oxide fuel cell and a molt...   
ENGI          748    747  Axial load-strain relationships of partially e...   
ENVI          529    529  Synergistic degradation of pyrene by five cult...   
HEAL          119    119  Is the occiput-wall distance valid and reliabl...   
IMMU          437    437  A cytosolic sensor, PmDDX41, mediates antivira...   
MATE          678    677  Structure and mechanical properties of ADC 12 ...   
MATH          208    208  Half-normal approximation for statistics of sy...   
MEDI         2839   2837  2020 APHRS/HRS expert consensus statement on t...   
MULT          264    264  Modelling and electrical characteristics of th...   
NEUR          214    214  Zika Virus as an Emerging Neuropathogen: Mecha...   
NURS          115    115  Temporal Trends Between 2010 and 2015 in Inten...   
PHAR          374    374  A novel flavanone derivative inhibits dengue v...   
PHYS          508    508  Effect of gamma irradiation on properties of u...   
PSYC          130    130  I'll get there because I'm great, or am I? Nar...   
SOCI          623    623  Design and development of SECI and moodle onli...   
VETE          295    295  Monitoring and controlling ovarian activity in...   

                 publicationName         \
            freq           count unique   
subjectArea                               
AGRI           1             899    289   
ARTS           1             171     63   
BIOC           2             934    323   
BUSI           1             280    146   
CENG           1             581    101   
CHEM           1             825    159   
COMP           2             870    277   
DECI           1              17     12   
DENT           1             233     70   
EART           1             183     79   
ECON           1             104     62   
ENER           1             391     82   
ENGI           2             748    250   
ENVI           1             529    146   
HEAL           1             119     51   
IMMU           1             437    108   
MATE           2             678    165   
MATH           1             208    103   
MEDI           2            2839   1045   
MULT           1             264     24   
NEUR           1             214     63   
NURS           1             115     54   
PHAR           1             374    108   
PHYS           1             508     90   
PSYC           1             130     64   
SOCI           1             623    291   
VETE           1             295     44   

                                                                    abstract  \
                                                           top freq    count   
subjectArea                                                                    
AGRI                                                 Nutrients   39      899   
ARTS         Nakhara: Journal 

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import compute_class_weight
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

# Assuming you have a DataFrame 'df' with columns: title, publicationName, abstract, keywords, subjectArea

# Text preprocessing and vectorization
text_data = df['title'] + ' ' + df['publicationName'] + ' ' + df['abstract'] + ' ' + df['keywords']
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text_data)
y = df['subjectArea']

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_encoded

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Create and train XGBoost model
model = XGBClassifier()
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.68      0.72      0.70       184
           1       0.83      0.48      0.61        31
           2       0.54      0.44      0.48       201
           3       0.62      0.47      0.53        49
           4       0.79      0.63      0.70       125
           5       0.60      0.81      0.69       144
           6       0.78      0.85      0.81       185
           7       0.00      0.00      0.00         1
           8       0.79      0.66      0.72        50
           9       0.82      0.50      0.62        28
          10       0.69      0.42      0.52        26
          11       0.63      0.60      0.62        73
          12       0.59      0.58      0.58       149
          13       0.61      0.60      0.61       112
          14       0.57      0.20      0.30        20
          15       0.71      0.52      0.60        90
          16       0.67      0.67      0.67       145
          17       0.85    